In [1]:
from __future__ import absolute_import, division, print_function

# Data Exploration

In this notebook I will explore the data and ascertain their structure. 

## Imports

In [2]:
import os
import pickle

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

## Functions

In [3]:
# Calculate distance given long/lat
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2. * asin(sqrt(a)) 
    r = 6371. # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [4]:
def impute_with_nearest(in_df,in_column):
    """
    Replace missing values in 'in_column' of the data frame 'in_df' 
    with values from the same day that are closest geographically. 
    """
    # Data frame of all the 'NaN's    
    df_nan = in_df[in_df[in_column].isnull()]
    if len(df_nan) > 0:
        # Same date, similar locations
        for i in range(len(df_nan)):
            tmp_date = df_nan['Date'].iloc[i]
            tmp_long = df_nan['Longitude'].iloc[i]
            tmp_lat  = df_nan['Latitude'].iloc[i]
            tmp_index = df_nan['Latitude'].index.tolist()[i]
            condition = (((in_df['Latitude'] != tmp_lat) |
                         (in_df['Longitude'] != tmp_long)) & 
                         (in_df['Date'] == tmp_date) & 
                         (in_df[in_column].notnull())) 
            # Create temporary data frame for convenient ordering
            new_df = in_df[['Longitude', 'Latitude']][condition]
            new_df['Distance'] = new_df['Longitude']
            for j in range(len(new_df)):
                new_df['Distance'].iloc[j] = haversine(tmp_long,
                                                       tmp_lat,
                                                       new_df['Longitude'].iloc[j],
                                                       new_df['Latitude'].iloc[j])
            # Distance to nearest measurement on same day
            tmp_df = new_df.sort_values(by='Distance', ascending=True)
            tmp_min_dist = tmp_df.iloc[0,2]
            tmp_min_index = tmp_df.index.tolist()[0]
            # Replace
            old_val = in_df[in_column].iloc[tmp_index]
            new_val = in_df[in_column].iloc[tmp_min_index]
            in_df[in_column].iloc[tmp_index] = new_val


## Data

In [5]:
cwd = os.getcwd()
data = os.path.join(cwd,'data','wheat-2013-supervised.csv')
df_2013 = pd.read_csv(data)
df_2013.head()

,CountyName,State,Latitude,Longitude,Date,apparentTemperatureMax,apparentTemperatureMin,cloudCover,dewPoint,humidity,...,precipTypeIsOther,pressure,temperatureMax,temperatureMin,visibility,windBearing,windSpeed,NDVI,DayInSeason,Yield
0,Adams,Washington,46.811686,-118.695237,11/30/2013 0:00,35.70,20.85,0.00,29.53,0.91,...,0,1027.13,35.70,27.48,2.46,214,1.18,134.110657,0,35.7
1,Adams,Washington,46.929839,-118.352109,11/30/2013 0:00,35.10,26.92,0.00,29.77,0.93,...,0,1026.87,35.10,26.92,2.83,166,1.01,131.506592,0,35.7
2,Adams,Washington,47.006888,-118.510160,11/30/2013 0:00,33.38,26.95,0.00,29.36,0.94,...,0,1026.88,33.38,26.95,2.95,158,1.03,131.472946,0,35.7
3,Adams,Washington,47.162342,-118.699677,11/30/2013 0:00,28.05,25.93,0.91,29.47,0.94,...,0,1026.37,33.19,27.17,2.89,153,1.84,131.288300,0,35.7
4,Adams,Washington,47.157512,-118.434056,11/30/2013 0:00,28.83,25.98,0.91,29.86,0.94,...,0,1026.19,33.85,27.07,2.97,156,1.85,131.288300,0,35.7


In [6]:
# Also read data for 2014
data = os.path.join(cwd,'data','wheat-2014-supervised.csv')
df_2014 = pd.read_csv(data)
df_2014.head()

,CountyName,State,Latitude,Longitude,Date,apparentTemperatureMax,apparentTemperatureMin,cloudCover,dewPoint,humidity,...,precipTypeIsOther,pressure,temperatureMax,temperatureMin,visibility,windBearing,windSpeed,NDVI,DayInSeason,Yield
0,Adams,Washington,46.929839,-118.352109,11/30/2014 0:00,18.61,-3.01,0.00,6.77,0.69,...,0,1027.95,23.93,6.96,10.00,9,3.80,136.179718,0,35.6
1,Adams,Washington,47.150327,-118.958859,11/30/2014 0:00,19.67,-0.74,0.00,6.66,0.65,...,0,1028.26,25.88,8.71,10.00,352,6.03,135.697540,0,35.6
2,Adams,Washington,46.811686,-118.695237,11/30/2014 0:00,20.66,-0.14,0.00,6.55,0.67,...,0,1028.29,24.67,8.26,10.00,25,3.59,135.676956,0,35.6
3,Adams,Washington,47.162342,-118.699677,11/30/2014 0:00,19.69,-2.66,0.03,7.32,0.69,...,0,1027.74,25.48,8.10,10.00,1,5.18,135.005798,0,35.6
4,Adams,Washington,47.157512,-118.434056,11/30/2014 0:00,18.82,-3.04,0.04,7.62,0.70,...,0,1027.16,24.83,8.32,9.99,5,4.69,134.803864,0,35.6


## NaN values

In [7]:
# Check the general properties of the DFs
df_2013.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177493 entries, 0 to 177492
Data columns (total 26 columns):
CountyName                177493 non-null object
State                     177493 non-null object
Latitude                  177493 non-null float64
Longitude                 177493 non-null float64
Date                      177493 non-null object
apparentTemperatureMax    177493 non-null float64
apparentTemperatureMin    177493 non-null float64
cloudCover                177493 non-null float64
dewPoint                  177493 non-null float64
humidity                  177493 non-null float64
precipIntensity           177492 non-null float64
precipIntensityMax        177492 non-null float64
precipProbability         177492 non-null float64
precipAccumulation        177493 non-null float64
precipTypeIsRain          177493 non-null int64
precipTypeIsSnow          177493 non-null int64
precipTypeIsOther         177493 non-null int64
pressure                  177239 non-null float6

In [8]:
df_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182549 entries, 0 to 182548
Data columns (total 26 columns):
CountyName                182549 non-null object
State                     182549 non-null object
Latitude                  182549 non-null float64
Longitude                 182549 non-null float64
Date                      182549 non-null object
apparentTemperatureMax    182549 non-null float64
apparentTemperatureMin    182549 non-null float64
cloudCover                182549 non-null float64
dewPoint                  182549 non-null float64
humidity                  182549 non-null float64
precipIntensity           182549 non-null float64
precipIntensityMax        182549 non-null float64
precipProbability         182549 non-null float64
precipAccumulation        182549 non-null float64
precipTypeIsRain          182549 non-null int64
precipTypeIsSnow          182549 non-null int64
precipTypeIsOther         182549 non-null int64
pressure                  182198 non-null float6

### For both years I have 26 columns with ~177k rows for 2013 and ~183k rows for 2014. All columns show the expected format for the data they contain (float, integer, string). However, for each year a number of columns have a few entries that are 'NaN' (compare number on non-null entries with total number of rows).

In [9]:
# Let's verify
df_2013.isnull().sum()

CountyName                  0
State                       0
Latitude                    0
Longitude                   0
Date                        0
apparentTemperatureMax      0
apparentTemperatureMin      0
cloudCover                  0
dewPoint                    0
humidity                    0
precipIntensity             1
precipIntensityMax          1
precipProbability           1
precipAccumulation          0
precipTypeIsRain            0
precipTypeIsSnow            0
precipTypeIsOther           0
pressure                  254
temperatureMax              0
temperatureMin              0
visibility                 30
windBearing                 0
windSpeed                   0
NDVI                        0
DayInSeason                 0
Yield                       0
dtype: int64

In [10]:
df_2014.isnull().sum()

CountyName                  0
State                       0
Latitude                    0
Longitude                   0
Date                        0
apparentTemperatureMax      0
apparentTemperatureMin      0
cloudCover                  0
dewPoint                    0
humidity                    0
precipIntensity             0
precipIntensityMax          0
precipProbability           0
precipAccumulation          0
precipTypeIsRain            0
precipTypeIsSnow            0
precipTypeIsOther           0
pressure                  351
temperatureMax              0
temperatureMin              0
visibility                 16
windBearing                 0
windSpeed                   0
NDVI                        0
DayInSeason                 0
Yield                       0
dtype: int64

### In 2013, five features have 'NaN' values, in 2014 only 2 features. The feature with the highest number of 'NaNs' is 'pressure' followed by 'visibilty' for both years. All missing values are from features that are weather related.

### Convert date column to datetime format

In [11]:
# Check if already converted b/c it takes a long time
if not df_2013['Date'].dtype == np.dtype('datetime64[ns]'):
    df_2013['Date'] = df_2013['Date'].apply(pd.to_datetime)

In [12]:
# Check if already converted b/c it takes a long time
if not df_2014['Date'].dtype == np.dtype('datetime64[ns]'):
    df_2014['Date'] = df_2014['Date'].apply(pd.to_datetime)

## Impute missing values

### Because the data are collected in very different areas, a global average can not be used for the imputed value (which are weather data, i.e. local). For each 'NaN' I will get the prior and the next value (according to date) at the same location (according to Latitude and Longitude) and replace the 'NaN' with the average of the other two values.

In [13]:
# Get df for all the NaN values
df_nan = df_2014[df_2014['pressure'].isnull()]

In [14]:
# Go through each entry with 'NaN' in pressure (as an example) and find the bracketing (in time) measure at this location
date_diff = []
for i in range(len(df_nan)):
    tmp_date = df_nan['Date'].iloc[i]
    tmp_long = df_nan['Longitude'].iloc[i]
    tmp_lat  = df_nan['Latitude'].iloc[i]
    condition = ((df_2014['Longitude'] == tmp_long) & 
                 (df_2014['Latitude'] == tmp_lat) & 
                 (df_2014['Date'] > tmp_date)) 
    if len(df_2014['Date'][condition]) > 0:
        date_diff.append(tmp_date-df_2014['Date'][condition].iloc[0])
    # Same location, later dates
    condition = ((df_2014['Longitude'] == tmp_long) & 
                 (df_2014['Latitude'] == tmp_lat) & 
                 (df_2014['Date'] < tmp_date)) 
    if len(df_2014['Date'][condition]) > 0:
        date_diff.append(tmp_date-df_2014['Date'][condition].iloc[0])

In [21]:
# Show min/max of the difference in days between the missing value and the next closest in time at the same location
date_diff_float = [abs(x.days) for x in date_diff]
print('Min: {} ; Max: {} ; Mean : {}'.format(min(date_diff_float),max(date_diff_float),np.mean(date_diff_float)))

Min: 4 ; Max: 169 ; Mean : 48.9235993209


### Dates seem to be too sparse to calculate a meaningful replacement value from measurements close in time. Instead use the measurements that are closest geographically and obtained on the same day.

### I see missing values in the following fields:

* 2013: precipIntensity, precipIntensityMax, precipProbability, pressure, visibility
* 2014: pressure, visibility     

In [22]:
impute_with_nearest(df_2013, 'precipIntensity')
impute_with_nearest(df_2013, 'precipIntensityMax')
impute_with_nearest(df_2013, 'precipProbability')
impute_with_nearest(df_2013, 'pressure')
impute_with_nearest(df_2013, 'visibility')
# #
impute_with_nearest(df_2014, 'pressure')
impute_with_nearest(df_2014, 'visibility')

In [23]:
# Check result
print(df_2013.isnull().sum().sum())
print(df_2014.isnull().sum().sum())

0
0


### Done!

## Basic statistics

In [24]:
X = df_2013.describe().T
X['missing %'] = 1 - (X['count'] / len(df_2013))
X

,count,mean,std,min,25%,50%,75%,max,missing %
Latitude,177493.0,37.525355,4.918148,27.803015,34.135464,36.810364,38.945878,48.981928,0.0
Longitude,177493.0,-100.877849,5.960510,-120.914093,-101.294945,-99.128028,-97.352044,-94.613571,0.0
apparentTemperatureMax,177493.0,54.836240,23.827278,-39.970000,37.830000,58.880000,73.100000,177.320000,0.0
apparentTemperatureMin,177493.0,27.915526,22.029381,-58.420000,14.310000,26.560000,42.200000,77.180000,0.0
cloudCover,177493.0,0.071478,0.138932,0.000000,0.000000,0.010000,0.090000,1.000000,0.0
dewPoint,177493.0,29.712311,16.687710,-36.090000,19.600000,27.850000,38.890000,75.180000,0.0
humidity,177493.0,0.593973,0.163590,0.080000,0.470000,0.600000,0.720000,1.000000,0.0
precipIntensity,177493.0,0.001158,0.004559,0.000000,0.000000,0.000000,0.000200,0.152900,0.0
precipIntensityMax,177493.0,0.010633,0.044782,0.000000,0.000000,0.000000,0.002800,2.054900,0.0
precipProbability,177493.0,0.133540,0.253415,0.000000,0.000000,0.000000,0.090000,0.960000,0.0


In [25]:
X = df_2014.describe().T
X['missing %'] = 1 - (X['count'] / len(df_2014))
X

,count,mean,std,min,25%,50%,75%,max,missing %
Latitude,182549.0,36.976937,4.926798,27.363741,33.598030,36.164529,38.451645,48.981928,0.0
Longitude,182549.0,-100.547088,5.587446,-120.914093,-101.174610,-99.068063,-97.270576,-94.164602,0.0
apparentTemperatureMax,182549.0,57.146379,20.217840,-24.340000,41.390000,61.460000,72.990000,103.910000,0.0
apparentTemperatureMin,182549.0,33.669343,19.649074,-43.250000,20.890000,33.310000,50.630000,79.560000,0.0
cloudCover,182549.0,0.131278,0.170087,0.000000,0.010000,0.080000,0.180000,1.000000,0.0
dewPoint,182549.0,36.664752,16.202934,-21.950000,25.740000,35.130000,48.840000,78.350000,0.0
humidity,182549.0,0.680338,0.150459,0.140000,0.570000,0.680000,0.800000,1.000000,0.0
precipIntensity,182549.0,0.002501,0.007232,0.000000,0.000000,0.000000,0.001000,0.255800,0.0
precipIntensityMax,182549.0,0.021390,0.071354,0.000000,0.000000,0.000000,0.010100,3.367400,0.0
precipProbability,182549.0,0.205662,0.307787,0.000000,0.000000,0.000000,0.510000,0.970000,0.0


### For both years, the (binary) column 'precipTypeIsOther' is always zero (zero std and zero min/max). This feature can be safely dropped as it holds no predictive power.

## Correlations

### Pairwise correlations between different features can help to eliminate features from modeling. Adding highly correlated features can dilute the predictive power of the model. 

In [26]:
# Find all column pairs with correlation coefficients above a certain threshold
X = df_2013.corr()
limit = 0.65
index = X.index.tolist()
n = len(index)
newDF = pd.DataFrame(columns=['col1','col2','corr','corr col1 with target', 'corr col2 with target'])
pos = 0
for nidx,idx in enumerate(index):
    for k in range(n):
        if (k > nidx):
            value = X[idx].iloc[k]
        else:
            value = 0.0
        if value >= limit:
            corr_col1 = X['Yield'].loc[idx]
            corr_col2 = X['Yield'].loc[index[k]]
            newDF.loc[pos] = [idx, index[k] , value, corr_col1, corr_col2]
            pos += 1


In [27]:
newDF.sort_values(by='corr', ascending=False)

,col1,col2,corr,corr col1 with target,corr col2 with target
7,apparentTemperatureMin,temperatureMin,0.992737,-0.049998,-0.060512
2,apparentTemperatureMax,temperatureMax,0.991843,-0.140315,-0.152702
12,precipIntensity,precipIntensityMax,0.912594,0.046476,0.030030
11,dewPoint,temperatureMin,0.910333,0.011607,-0.060512
5,apparentTemperatureMin,dewPoint,0.909563,-0.049998,0.011607
6,apparentTemperatureMin,temperatureMax,0.893452,-0.049998,-0.152702
0,apparentTemperatureMax,apparentTemperatureMin,0.888862,-0.140315,-0.049998
14,temperatureMax,temperatureMin,0.886946,-0.152702,-0.060512
3,apparentTemperatureMax,temperatureMin,0.880383,-0.140315,-0.060512
1,apparentTemperatureMax,dewPoint,0.796342,-0.140315,0.011607


In [28]:
# Find all column pairs with correlation coefficients above a certain threshold
X = df_2014.corr()
limit = 0.65
index = X.index.tolist()
n = len(index)
newDF = pd.DataFrame(columns=['col1','col2','corr','corr col1 with target', 'corr col2 with target'])
pos = 0
for nidx,idx in enumerate(index):
    for k in range(n):
        if (k > nidx):
            value = X[idx].iloc[k]
        else:
            value = 0.0
        if value >= limit:
            corr_col1 = X['Yield'].loc[idx]
            corr_col2 = X['Yield'].loc[index[k]]
            newDF.loc[pos] = [idx, index[k] , value, corr_col1, corr_col2]
            pos += 1


In [29]:
newDF.sort_values(by='corr', ascending=False)

,col1,col2,corr,corr col1 with target,corr col2 with target
2,apparentTemperatureMax,temperatureMax,0.993237,-0.165742,-0.172160
7,apparentTemperatureMin,temperatureMin,0.992886,-0.146381,-0.153430
9,dewPoint,temperatureMin,0.951772,-0.158787,-0.153430
5,apparentTemperatureMin,dewPoint,0.949988,-0.146381,-0.158787
10,precipIntensity,precipIntensityMax,0.863038,-0.015459,-0.020094
0,apparentTemperatureMax,apparentTemperatureMin,0.849104,-0.165742,-0.146381
6,apparentTemperatureMin,temperatureMax,0.844614,-0.146381,-0.172160
3,apparentTemperatureMax,temperatureMin,0.841444,-0.165742,-0.153430
13,temperatureMax,temperatureMin,0.838178,-0.172160,-0.153430
1,apparentTemperatureMax,dewPoint,0.796485,-0.165742,-0.158787


### There are a few features that show very strong correlation. For some that is to be expected (apparent temperature vs. temperature, precip intensity vs. max precip intensity, etc).

### The 'apparent' temperature values are slightly less correlated with the target variable ('Yield'). I will discard those from further analysis. While 'dew point' and 'temperature min' are highly correlated, thier correlation with 'Yield' has opposite sign (even though the linear correlation itself is weak). I will keep both for now.

## Drop features/columns

- precipTypeIsOther
- apparentTemperatureMin
- apparentTemperatureMax
- precipIntensityMax


In [30]:
df_2013.drop('precipTypeIsOther', axis=1, inplace=True)
df_2013.drop('apparentTemperatureMin', axis=1, inplace=True)
df_2013.drop('apparentTemperatureMax', axis=1, inplace=True)
df_2013.drop('precipIntensityMax', axis=1, inplace=True)
#
df_2014.drop('precipTypeIsOther', axis=1, inplace=True)
df_2014.drop('apparentTemperatureMin', axis=1, inplace=True)
df_2014.drop('apparentTemperatureMax', axis=1, inplace=True)
df_2014.drop('precipIntensityMax', axis=1, inplace=True)


For additional information on the 'NDVI' parameter: "The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, typically but not necessarily from a space platform, and assess whether the target being observed contains live green vegetation or not." (source: Wikipedia).

## Save cleaned data to disk

In [31]:
df_2013.to_pickle(os.path.join('data','df_2013_clean.df'))
df_2014.to_pickle(os.path.join('data','df_2014_clean.df'))

